# Hadoop - Decommission DataNode

---

Slave NodeのHDDの故障時などに、Slave NodeをDecommissionするための手順です。

我々のオペレーションでは、DataNodeのDecommission後、マシンを再起動するため、YARNのNodeManager, HBaseのRegionServerもあわせて停止しています。

## *Operation Note*

*This is a cell for your own recording. ここに経緯を記述*

# Notebookと対象(Hadoopクラスタ)のBinding

Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'hadoop_all_testcluster'

処理を実行するホストの情報を指示する。

In [2]:
# 状態を確認する処理を行うNameNode
client_host = 'hadoop_namenode_primary'

# デコミッションする対象のDataNode
target_host = "XXX.XXX.XXX.114"

# DataNodeのデコミッション

## デコミッションしたいノードの情報を確認する

デコミッションしたいノードのNameを確認する

In [5]:
!ansible {client_host} -a "hdfs dfsadmin -report" --become --become-user hdfs -l { target_group }

XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
Configured Capacity: 422216597504 (393.22 GB)
Present Capacity: 389017650397 (362.30 GB)
DFS Remaining: 388825679069 (362.12 GB)
DFS Used: 191971328 (183.08 MB)
DFS Used%: 0.05%
Under replicated blocks: 10
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 0

-------------------------------------------------
Live datanodes (4):

Name: XXX.XXX.XXX.114:50010 (testvm007)
Hostname: testvm007
Decommission Status : Normal
Configured Capacity: 105554149376 (98.30 GB)
DFS Used: 3747840 (3.57 MB)
Non DFS Used: 8198860551 (7.64 GB)
DFS Remaining: 97351540985 (90.67 GB)
DFS Used%: 0.00%
DFS Remaining%: 92.23%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 7
Last contact: Mon Aug 22 11:12:55 JST 2016


Name: XXX.XXX.XXX.73:50010 (testvm004)
Hostname: testvm004
Decommission Status : Normal
Configured Capacity: 105554149376 (98.30 GB)
D

デコミッションしたいノードの Name: の欄を確認する

## デコミッションしたいノード情報をgroup_varsに追記する

デコミッション対象のノードリストを作成する。

ansibleのユーザ変数の *datanode_decommission_nodes* に上で確認した *Name:* を記載する

記載例

    datanode_decommission_nodes:
    - XXX.XXX.XXX.114

In [6]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpbSg7h_'

現在の設定ファイル(`group_vars/{target_group}`)のバックアップを取っておく。 

In [8]:
!cp group_vars/{target_group} {work_dir}/{target_group}-old

[hadoop\_all\_{target_group}](../tree/group_vars/) に以下のコードを追加する。(すでにDecommissionしているノードがあればリストに追加する)

In [9]:
print('datanode_decommission_nodes:\n  - {}\n'.format(target_host))

datanode_decommission_nodes:
  - XXX.XXX.XXX.114



In [10]:
!diff -ur {work_dir}/{target_group}-old group_vars/{target_group}

--- /tmp/tmpbSg7h_/hadoop_all_testcluster-old	2016-08-22 11:14:48.786247539 +0900
+++ group_vars/hadoop_all_testcluster	2016-08-22 11:17:48.855760027 +0900
@@ -84,3 +84,5 @@
 swimlane_tez_version: 0.8.4
 swimlane_tmp_path: /tmp
 
+datanode_decommission_nodes:
+  - XXX.XXX.XXX.114
\ No newline at end of file


## Hadoopのコンフィグファイルを更新する

上で変更した設定をHadoopのコンフィグファイルに反映させる。

まず、Playbookを準備する。

In [11]:
!rm -fr {work_dir}/hadoop
!git clone https://github.com/NII-cloud-operation/Literate-computing-Hadoop.git {work_dir}/hadoop
!tree {work_dir}/hadoop

Cloning into '/tmp/tmpbSg7h_/hadoop'...
remote: Counting objects: 841, done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 841 (delta 88), reused 0 (delta 0), pack-reused 527
Receiving objects: 100% (841/841), 184.13 KiB | 294.00 KiB/s, done.
Resolving deltas: 100% (181/181), done.
Checking connectivity... done.
/tmp/tmpbSg7h_/hadoop
└── playbooks
    ├── conf_base.retry
    ├── conf_base.yml
    ├── conf_hdfs_base.yml
    ├── conf_hdfs_spark.yml
    ├── conf_hdfs_tez.yml
    ├── conf_hdfs_yarn.yml
    ├── conf_namenode_bootstrapstandby.yml
    ├── conf_tez.yml
    ├── enter_hdfs_safemode.yml
    ├── format_namenode.yml
    ├── group_vars
    │   └── all
    │       ├── base
    │       ├── cgroups
    │       ├── collect
    │       ├── f500.dumpall
    │       ├── hbase_master
    │       ├── hbase_regionserver
    │       ├── hcatalog
    │       ├── hdfs_base
    │       ├── hdfs_spark
    │       ├── hdfs_tez
    │       ├── hdfs_yarn
    │       ├── hive
    │ 

In [13]:
import os
playbook_dir = os.path.join(work_dir, 'hadoop/playbooks')

Playbookを準備したら、configの更新を実施する・・・まずはdry-runしてみる。

`/etc/hadoop/conf/hosts.exclude` にDecommissionしたいSlave Nodeのアドレスが追加されることを確認する。

In [14]:
!ansible-playbook -CDv {playbook_dir}/conf_base.yml -l {target_group}

Using /etc/ansible/ansible.cfg as config file

PLAY [hadoop_all] **************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.114]

TASK [base : include] **********************************************************
included: /tmp/tmpbSg7h_/hadoop/playbooks/roles/base/tasks/repo.yml for XXX.XXX.XXX.70, XXX.XXX.XXX.71, XXX.XXX.XXX.72, XXX.XXX.XXX.73, XXX.XXX.XXX.112, XXX.XXX.XXX.113, XXX.XXX.XXX.114

TASK [base : install_hdp_repo] *************************************************
ok: [XXX.XXX.XXX.72] => {"changed": false, "gid": 0, "group": "root", "mode": "0644", "owner": "root", "path": "/etc/yum.repos.d/hdp.repo", "size": 556, "state": "file", "uid": 0}
ok: [XXX.XXX.XXX.71] => {"changed": false, "gid": 0, "group": "root", "mode": "0644", "owner": "root", "path":

変更内容を確認し、意図しない変更が含まれていなければ、設定変更を適用する。

In [15]:
!ansible-playbook {playbook_dir}/conf_base.yml -l {target_group}


PLAY [hadoop_all] **************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.114]

TASK [base : include] **********************************************************
included: /tmp/tmpbSg7h_/hadoop/playbooks/roles/base/tasks/repo.yml for XXX.XXX.XXX.70, XXX.XXX.XXX.71, XXX.XXX.XXX.72, XXX.XXX.XXX.73, XXX.XXX.XXX.112, XXX.XXX.XXX.113, XXX.XXX.XXX.114

TASK [base : install_hdp_repo] *************************************************
ok: [XXX.XXX.XXX.72]
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.114]

TASK [base : include] **********************************************************
included: /tmp/tmpbSg7h_/hadoop/playbooks/roles/base/tasks/conf.yml for XXX.XXX.XXX.70, XXX.XXX.XXX.71, 

## デコミッション設定を反映させる

更新したHadoopコンフィグファイルの情報をNameNodeに反映させる

In [16]:
!ansible hadoop_namenode -m shell -a 'hdfs dfsadmin -fs hdfs://$(hostname):8020 -refreshNodes' -b --become-user hdfs -l {target_group}

XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
Refresh nodes successful

XXX.XXX.XXX.71 | SUCCESS | rc=0 >>
Refresh nodes successful



各ノードから *Refresh nodes successful* が表示されることを確認する

このコマンドが実行されるとDataNodeのデコミッション処理が開始される

## デコミッションが行われたことを確認する

Decommissionの実施状況を確認する。

In [18]:
!ansible -m shell -a "hdfs dfsadmin -report" --become --become-user hdfs { client_host } -l {target_group}

XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
Configured Capacity: 316666195968 (294.92 GB)
Present Capacity: 291664450118 (271.63 GB)
DFS Remaining: 291470423012 (271.45 GB)
DFS Used: 194027106 (185.04 MB)
DFS Used%: 0.07%
Under replicated blocks: 10
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 0

-------------------------------------------------
Live datanodes (4):

Name: XXX.XXX.XXX.114:50010 (testvm007)
Hostname: testvm007
Decommission Status : Decommission in progress
Configured Capacity: 105554149376 (98.30 GB)
DFS Used: 3747840 (3.57 MB)
Non DFS Used: 8199270151 (7.64 GB)
DFS Remaining: 97351131385 (90.67 GB)
DFS Used%: 0.00%
DFS Remaining%: 92.23%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 7
Last contact: Mon Aug 22 11:24:07 JST 2016


Name: XXX.XXX.XXX.73:50010 (testvm004)
Hostname: testvm004
Decommission Status : Normal
Configured Capacity: 1055541

デコミッション対象のDecommission Statusが *Decommissioned* になっていることを確認する

+ *Decommision on progress* の場合は、処理が進行中であることを意味する。しばらくたってから再度確認する

In [19]:
!ansible -m shell -a "hdfs dfsadmin -report" --sudo --sudo-user hdfs { client_host } -l {target_group}

XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
Configured Capacity: 316666195968 (294.92 GB)
Present Capacity: 291664720454 (271.63 GB)
DFS Remaining: 291470693348 (271.45 GB)
DFS Used: 194027106 (185.04 MB)
DFS Used%: 0.07%
Under replicated blocks: 10
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 0

-------------------------------------------------
Live datanodes (4):

Name: XXX.XXX.XXX.114:50010 (testvm007)
Hostname: testvm007
Decommission Status : Decommissioned
Configured Capacity: 105554149376 (98.30 GB)
DFS Used: 3747840 (3.57 MB)
Non DFS Used: 8199261959 (7.64 GB)
DFS Remaining: 97351139577 (90.67 GB)
DFS Used%: 0.00%
DFS Remaining%: 92.23%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 7
Last contact: Mon Aug 22 11:24:46 JST 2016


Name: XXX.XXX.XXX.73:50010 (testvm004)
Hostname: testvm004
Decommission Status : Normal
Configured Capacity: 105554149376 (98.

## DataNodeを停止する

デコミッションが完了したDataNodeを停止させる

In [22]:
!ansible-playbook {playbook_dir}/stop_datanode.yml -l { target_host }


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.114]

TASK [stop_hadoop-hdfs-datanode] ***********************************************
changed: [XXX.XXX.XXX.114]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   



# NodeManagerを停止する

今回はSlave Nodeとしての機能をすべて停止する・・・

In [23]:
!ansible-playbook {playbook_dir}/stop_nodemanager.yml -l { target_host }


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.114]

TASK [stop_hadoop-yarn-nodemanager] ********************************************
changed: [XXX.XXX.XXX.114]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   



# RegionServerを停止する

In [24]:
!ansible-playbook {playbook_dir}/stop_hbase_regionserver.yml -l { target_host }


PLAY [hadoop_hbase_regionserver] ***********************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.114]

TASK [stop_hbase-regionserver] *************************************************
changed: [XXX.XXX.XXX.114]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   



# fsckしてみる

In [25]:
!ansible hadoop_client -b -U hdfs -a 'hdfs fsck /' -l {target_group}

XXX.XXX.XXX.72 | SUCCESS | rc=0 >>
FSCK started by hdfs (auth:SIMPLE) from /XXX.XXX.XXX.72 for path / at Mon Aug 22 11:27:04 JST 2016
..............
/tmp/hadoop-yarn/staging/ansible/.staging/job_1471559811619_0001/job.jar:  Under replicated BP-975593033-XXX.XXX.XXX.70-1471530072968:blk_1073741935_1111. Target Replicas is 10 but found 3 live replica(s), 1 decommissioned replica(s) and 0 decommissioning replica(s).
.
/tmp/hadoop-yarn/staging/ansible/.staging/job_1471559811619_0001/job.split:  Under replicated BP-975593033-XXX.XXX.XXX.70-1471530072968:blk_1073741936_1112. Target Replicas is 10 but found 3 live replica(s), 1 decommissioned replica(s) and 0 decommissioning replica(s).
...
/tmp/hadoop-yarn/staging/ansible/.staging/job_1471559811619_0002/job.jar:  Under replicated BP-975593033-XXX.XXX.XXX.70-1471530072968:blk_1073741960_1136. Target Replicas is 10 but found 3 live replica(s), 1 decommissioned replica(s) and 0 decommissioning replica(s).
.
/tmp/hadoop-yarn/staging/ansible/.sta

# 後始末

一時ディレクトリを削除する。

In [26]:
!rm -fr {work_dir}